#  Perrón (1989): "The Great Crash, the Oil Price Shock, and the Unit Root Hypothesis" 

In [2]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import acf
from statsmodels.formula.api import ols
from pandas import ExcelWriter


In [3]:
# Upload data
datos = pd.read_excel("/Users/lucassalamanca/Downloads/data.xlsx")
datos.set_index('Year', inplace=True)


In [4]:
# Renanme columns and apply logarithms
new_column_names = ['lrgnp', 'lgnp', 'lpcrgnp', 'lip', 'lemp', 'lun', 'lprgnp', 'lcpi', 'lwg', 'lrwg', 'lm', 'lvel', 'bnd', 'lsp500']
columns_to_log = datos.columns[datos.columns != datos.columns[12]]
datos[columns_to_log] = datos[columns_to_log].apply(np.log)
datos.columns = new_column_names

In [5]:
# Mapear nombres descriptivos
variables = {
    'lrgnp':'Real GNP',
    'lgnp':'Nominal GNP',
    'lpcrgnp':'Real per capita GNP',
    'lip':'Industrial production',
    'lemp':'Employment', 
    'lun':'Unemployment rate',
    'lprgnp':'GNP deflator',
    'lcpi':'Consumer prices',
    'lwg':'Wages',
    'lrwg':'Real wages', 
    'lm':'Money stock', 
    'lvel':'Velocity',
    'bnd':'Bond yield',
    'lsp500':'Common stock prices'
}

datos.columns = [variables[col] for col in datos.columns]

# Build matrix with the critical values for Model A 
lambda_values_A = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
valores_criticos_A = np.array([
    [-4.30, -3.68, -3.40],
    [-4.39, -3.77, -3.47],
    [-4.39, -3.76, -3.46],
    [-4.34, -3.72, -3.44],
    [-4.32, -3.76, -3.46],
    [-4.45, -3.76, -3.47],
    [-4.42, -3.80, -3.51],
    [-4.33, -3.75, -3.46],
    [-4.27, -3.69, -3.38]
])
valores_criticos_A_df = pd.DataFrame(valores_criticos_A, columns=['1%', '5%', '10%'], index=lambda_values_A)

# Build matrix with the critial values for Model C 
lambda_values_C = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
valores_criticos_C = np.array([
    [-4.38, -3.75, -3.45],
    [-4.65, -3.99, -3.66],
    [-4.78, -4.17, -3.87],
    [-4.81, -4.22, -3.95],
    [-4.90, -4.24, -3.96],
    [-4.88, -4.24, -3.95],
    [-4.75, -4.18, -3.86],
    [-4.70, -4.04, -3.69],
    [-4.41, -3.80, -3.46]
])
valores_criticos_C_df = pd.DataFrame(valores_criticos_C, columns=['1%', '5%', '10%'], index=lambda_values_C)

# Dictionary with specific lag number for each series 
rezagos = {
    'Real GNP': 8,
    'Nominal GNP': 8,
    'Real per capita GNP': 7,
    'Industrial production': 8,
    'Employment': 7,
    'GNP deflator': 5,
    'Consumer prices': 2,
    'Wages': 7,
    'Money stock': 6,
    'Velocity': 0,
    'Bond yield': 3,
    'Real wages': 8,
    'Common stock prices': 1,
}

# Dictionary with specific lambda values for each series 
valores_lambda_A = {
    'Real GNP': 0.33,
    'Nominal GNP': 0.33,
    'Real per capita GNP': 0.33,
    'Industrial production': 0.63,
    'Employment': 0.49,
    'GNP deflator': 0.49,
    'Consumer prices': 0.63,
    'Wages': 0.41,
    'Money stock': 0.49,
    'Velocity': 0.59,
    'Bond yield': 0.41,
    'Real wages': 0.41,
    'Common stock prices': 0.59
}

In [6]:
# Function that retrieves critical values for Model A 
def obtener_valores_criticos_A(lamda):
    closest_index = lambda_values_A[lambda_values_A <= lamda][-1]
    valores_criticos = valores_criticos_A_df.loc[closest_index]
    return {'1%': valores_criticos['1%'], '5%': valores_criticos['5%'], '10%': valores_criticos['10%']}

# Function that retrieves critical values for Model C
def obtener_valores_criticos_C(lamda):
    closest_index = lambda_values_C[lambda_values_C <= lamda][-1]
    valores_criticos = valores_criticos_C_df.loc[closest_index]
    return {'1%': valores_criticos['1%'], '5%': valores_criticos['5%'], '10%': valores_criticos['10%']}

Model A: includes a change in intercept

In [7]:
# Define function for Model A 
def Perrontest_A(serie, k):
    # Creamos tipos de choques para las hipótesis
    dta = datos[[serie]].dropna()
    dta.rename(columns={serie: 'y'}, inplace=True)
    dta['DL'] = (dta.index > 1929).astype(int)
    dta['DP'] = (dta.index == 1930).astype(int)
    dta['t'] = np.arange(dta.shape[0])
    dta['Ly'] = dta['y'].shift(1)
    dta['Dy'] = dta['y'].diff(1)

    for j in range(1, k+1):
        dta[f'D{j}y'] = dta['Dy'].shift(j)

    lags = ' + '.join(dta.columns[-k:])

    formula = f'y ~ DL + DP + t + Ly + {lags}'
    modelo = ols(formula, data=dta).fit()
    tval = ((modelo.params['Ly'] - 1) / modelo.bse['Ly']).round(2)
    lamda = valores_lambda_A[serie]
    crit = obtener_valores_criticos_A(lamda)
    
    print(f"Resultados de la regresión para la serie: {serie}")
    print(modelo.summary())
    
    return {
        '$t$': tval,
        '$\lambda$': lamda,
        '1%': crit['1%'],
        '5%': crit['5%'],
        '10%': crit['10%'],
        'data': dta  # Añadimos los datos al resultado
    }


Model C: includes a change in intercept and slope

In [8]:
# Define function for Model C
def Perrontest_C(serie, k):
    # Creamos tipos de choques para las hipótesis
    dta = datos[[serie]].dropna()
    dta.rename(columns={serie: 'y'}, inplace=True)
    dta['DL'] = (dta.index > 1929).astype(int)
    dta['DP'] = (dta.index == 1930).astype(int)
    dta['t'] = np.arange(dta.shape[0])
    dta['NewSlope'] = dta['t'] * dta['DL']
    dta['Ly'] = dta['y'].shift(1)
    dta['Dy'] = dta['y'].diff(1)

    for j in range(1, k+1):
        dta[f'D{j}y'] = dta['Dy'].shift(j)

    lags = ' + '.join(dta.columns[-k:])

    formula = f'y ~ DL + DP + t + NewSlope + Ly + {lags}'
    modelo = ols(formula, data=dta).fit()
    tval = ((modelo.params['Ly'] - 1) / modelo.bse['Ly']).round(2)
    lamda = valores_lambda_A[serie]
    crit = obtener_valores_criticos_C(lamda)
    
    print(f"Resultados de la regresión para la serie: {serie}")
    print(modelo.summary())
    
    return {
        '$t$': tval,
        '$\lambda$': lamda,
        '1%': crit['1%'],
        '5%': crit['5%'],
        '10%': crit['10%'],
        'data': dta  # Añadimos los datos al resultado
    }

In [9]:
# Create results table excluding Unemployment rate  
excluir_series = ['Unemployment rate']
resultados = []
datos_regresiones = {}

for var in variables.values():
    if var not in excluir_series:
        k = rezagos[var]  # Obtains number of lags for each series
        if var in ['Real wages', 'Common stock prices']:
            resultado_C = Perrontest_C(var, k)
            resultado_C['Variable'] = var
            resultado_C['Modelo'] = 'C'
            resultados.append(resultado_C)
            datos_regresiones[var] = {'C': resultado_C['data']}
        else:
            resultado_A = Perrontest_A(var, k)
            resultado_A['Variable'] = var
            resultado_A['Modelo'] = 'A'
            resultados.append(resultado_A)
            datos_regresiones[var] = {'A': resultado_A['data']}  # Save regression's data

tabla_resultados = pd.DataFrame(resultados).set_index(['Variable', 'Modelo'])

# Save results table in an Excel file 
with ExcelWriter("resultados_perronAyC.xlsx") as writer:
    tabla_resultados.drop(columns='data').to_excel(writer, sheet_name='Resultados')
    

# Show results table
print(tabla_resultados)

Resultados de la regresión para la serie: Real GNP
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.993
Model:                            OLS   Adj. R-squared:                  0.991
Method:                 Least Squares   F-statistic:                     470.0
Date:                Mon, 26 Aug 2024   Prob (F-statistic):           4.63e-39
Time:                        21:13:31   Log-Likelihood:                 90.118
No. Observations:                  53   AIC:                            -154.2
Df Residuals:                      40   BIC:                            -128.6
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
I